In [1]:
import numpy as np
from rich.progress import track

class Tanh:
    def activate(self, x):
        return np.tanh(x)
    
    def derivative(self, x):
        return 1 - self.activate(x) ** 2


class Layer:
    def __init__(self, len_inputs, neurons, function, last=False):
        shape = neurons, len_inputs + 1
        self.weights = np.random.uniform(-0.5, 0.5, size=shape)
        self.f = function
        self.last = last
        self.idx = None
        self.neurons = neurons
        self.len_inputs = len_inputs
    
    def forward(self, layer_input):
        self.input = layer_input
        self.net = self.input.dot(self.weights.T)
        self.output = self.f.activate(self.net)
        return self.output
    
    def backward(self, target, alpha, previous_delta=None, previous_weigth=None):
        if self.last:
            self.delta = (target - self.output) * self.f.derivative(self.net)
        else:
            self.delta = (np.delete(previous_delta.dot(previous_weigth).T, 0) * self.f.derivative(self.net))
        
        self.weights += np.array([self.delta]).T * np.array([self.input]) * alpha
        
        return self.delta, self.weights
        
        
        
        
    def __repr__(self):
        return f"({self.idx}º Layer, Neurons: {self.neurons}, Last: {self.last})"


class NeuralNetwork:
    def __init__(self, *layers: Layer):
        self.layers = list(layers)
        for idx, layer in enumerate(self.layers):
            layer.idx = idx + 1
        self.layers[-1].last = True
        self.len_inputs = self.layers[0].len_inputs
        
    def __repr__(self):
        return f"NeuralNetwork (Num_Layers: {len(self.layers)}, Len_Inputs: {self.len_inputs}, Layers: {self.layers})"
    
    @property
    def weights(self):
        resp = []
        for idx, layer in enumerate(self.layers):
            resp.append((idx+1, layer.weights))
        return resp
        
    def _forward(self, x_input):
        #input_layer = x_input
        input_layer = np.append(1, x_input)
        for layer in self.layers:
            out_layer = layer.forward(input_layer)
            input_layer = np.append(1, out_layer)
            
        return out_layer
    
    def _backward(self, y, alpha):
        for layer in reversed(self.layers):
            if layer.last:
                previous_delta, previous_weigth = layer.backward(y, alpha)
            else:
                previous_delta, previous_weigth = layer.backward(y, alpha, previous_delta, previous_weigth)
    
    def fit(self, x_train, y_train, epochs=2000, alpha=0.05):

        for epoch in track(range(epochs), description="Processing..."):
            outputs = []
            for x, y in zip(x_train, y_train):
                out = self._forward(x)
                self._backward(y, alpha)
                outputs.append(out)
                
            errors = np.array([sum(error) for error in (y_train - outputs) ** 2])
            self.mean_squared_error = sum(errors) / len(errors)
            
            if not epoch % 100:
                print(f"MSE: {self.mean_squared_error}")
                
                
    def predict(self, x):
        out = self._forward(x)
        return out

In [2]:
import polars as pl
import matplotlib.pyplot as plt

In [25]:
def number_to_neurons(n):
    res = [-1] * 10
    res[n] = 1
    return res

data_train = pl.read_csv("train.csv")

y_train = np.array(data_train.drop_in_place("label"))
y_train = np.array([number_to_neurons(y) for y in y_train])

x_train = np.array([row for row in data_train.rows()]) / 255

In [28]:
rede = NeuralNetwork(
    Layer(len_inputs=784, neurons=28, function=Tanh()),
    Layer(len_inputs=28, neurons=10, function=Tanh()),
)
rede.fit(x_train, y_train, epochs=50)

Output()

MSE: 1.0254692630158078

In [29]:
def evaluate(rede, x, y, total, inicial=0):
    points = 0
    for idx in range(1000,total):
        correct = np.argmax(y[idx])
        predict = np.argmax(rede.predict(x[idx]))
        if correct == predict:
            points += 1

    return points/total * 100

In [47]:
evaluate(rede, x_train, y_train, 42_000)

95.21666666666667

In [45]:
def save_weights(rede, n_cam):
    with open("weights.npy", "wb") as f:
        for idx in range(n_cam):
            np.save(f, rede.layers[idx].weights)

def load_weights(rede, n_cam):
    with open("weights.npy", "rb") as f:
        for idx in range(n_cam):
            rede.layers[idx].weights = np.load(f)

In [46]:
load_weights(rede, 2)

In [42]:
def kaggle_predict(rede, n):
    data_test = pl.read_csv("test.csv")
    x_test = np.array([row for row in data_test.rows()]) / 255
    kaggle_df = pl.read_csv("sample_submission.csv")
    predicts = []
    for idx in range(28_000):
        predict = np.argmax(rede.predict(x_test[idx]))
        predicts.append(predict)

    df_predicts = pl.DataFrame({
        "Label": predicts
    })

    submission = kaggle_df.update(df_predicts)
    submission.write_csv(f"predicts_kaggle_{n}.csv")


In [50]:
kaggle_predict(rede, 4)

In [51]:
import gradio as gr

def predizer_garrancho(garrancho):
    if garrancho is not None:
        x = garrancho.reshape(1, 784) / 255
        return np.argmax(rede.predict(x))
    

gr.Interface(fn=predizer_garrancho, 
             inputs="sketchpad",
             outputs="textbox",
             ).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [66]:
x = x_train
y = y_train
data = list(zip(x_train,y_train))
np.random.shuffle(data)
x_train,y_train = zip(*data)
np.array(y)

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1,  1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [ 1, -1, -1, ..., -1, -1, -1],
       [-1, -1,  1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])